# Goals

This is simply a data scraping notebook and can be safely ignored, as I have already collected the data.

In general, I find that development in notebooks can be extremely satisfying, especially for tasks like data scraping which require a lot of analysis and step-by-step progression.

# Guardian

In [17]:
import requests
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import time
from tqdm.auto import tqdm

In [1]:
index_url = 'https://www.theguardian.com/guardian-observer-style-guide-a'

In [18]:
r = requests.get(index_url)
html = r.content.decode()

In [16]:
# display(HTML(html))
sel = '#maincontent > div > p:nth-child(1)'

In [30]:
style_guide_links = BeautifulSoup(html).select_one(sel).find_all('a')
style_guide_links = list(map(lambda x: x['href'], style_guide_links))

In [33]:
page_html = {}
for l in tqdm(style_guide_links):
    r = requests.get(l)
    if r.status_code == 200:
        page_html[l] = r.content.decode()
    time.sleep(1)

  0%|          | 0/26 [00:00<?, ?it/s]

In [39]:
guardian_htmls = list(page_html.values())

In [ ]:
guardian_soups = list(map(BeautifulSoup, guardian_htmls))
all_paragraphs = list(map(lambda x: x.find_all(attrs={'class': 'dcr-1myha1i'}), guardian_soups))
to_filter = all_paragraphs[0][0]
all_paragraphs = list(map(lambda x: list(filter(lambda y: y != to_filter, x)), all_paragraphs))

In [90]:
import pyperclip
pyperclip.copy('\n'.join(list(map(str, all_paragraphs[0][0:13]))))

In [95]:
all_paragraphs[0]

[<p class="dcr-1myha1i"><strong>a or an before H?</strong><br/>Use an before a silent H: an heir, an hour, an honest politician, an honorary consul; use a before an aspirated H: a hero, a hotel, a historian (but don’t change a direct quote if the speaker says, for example, “an historic”). With abbreviations, be guided by pronunciation: eg an LSE student</p>,
 <p class="dcr-1myha1i"><strong>A*</strong><br/>(A-level and GCSE) not A-star</p>,
 <p class="dcr-1myha1i"><strong>A&amp;E</strong><br/>accident and emergency; in the US, it’s ER (emergency room)</p>,
 <p class="dcr-1myha1i"><strong>abattoir</strong></p>,
 <p class="dcr-1myha1i"><strong>Abbas, Mahmoud</strong><br/>president of the Palestinian Authority</p>,
 <p class="dcr-1myha1i"><strong>abbeys</strong><br/>take initial cap, eg Westminster Abbey</p>,
 <p class="dcr-1myha1i"><strong>Abbottabad</strong><br/>city in Pakistan where Osama bin Laden lived and died</p>,
 <p class="dcr-1myha1i"><strong>abbreviations and acronyms</strong><

In [170]:
# Iterate over each paragraph tag
guardian_rules = {}
for pars in all_paragraphs:
    for p in pars:
        if p.strong and p.strong == p.contents[0]:  # Check if paragraph starts with a strong tag
            current_title = p.strong.get_text().strip()  # Extract title
            guardian_rules[current_title] = p.get_text(separator=" ").replace(current_title, '', 1).strip()  # Initialize title in dictionary
        elif current_title:
            # Add text to the current title key
            guardian_rules[current_title] += "\n\n" + p.get_text(separator=" ").strip()

In [171]:
import pandas as pd 
guardian_rules_df = pd.Series(guardian_rules).to_frame('rule_text').reset_index().rename(columns={'index': 'rule_title'})

In [172]:
guardian_rules_df['rule_text'] = guardian_rules_df['rule_text'].str.replace(' +', ' ', regex=True)

In [173]:
# mkdir ../corpora/guardian

In [174]:
guardian_rules_df.to_csv('../corpora/guardian/parsed-rules-df.csv')
import pickle
with open('../corpora/guardian/all-htmls.pkl', 'wb') as f:
    pickle.dump(guardian_htmls, f)

In [175]:
guardian_rules_df['rule_text'].iloc[4]

'president of the Palestinian Authority'

In [176]:
print(guardian_rules_df.loc[lambda df: df['rule_title']== 'abortion']['rule_text'].iloc[0])

Use pro-choice , not pro-abortion; and use anti-abortion , not pro-life. Anti-choice can be used when talking about opposition to all reproductive rights, including abortion, birth control, family planning etc.

In reference to US legislation, six-week abortion ban is preferable to foetal heartbeat bill, unless quoting someone talking about the “heartbeat bill”


# Buzzfeed

In [265]:
t = 'https://www.buzzfeed.com/buzzfeednews/buzzfeed-style-guide'
r = requests.get(t)

In [266]:
# mkdir ../corpora/buzzfeed

In [267]:
html = r.content.decode()
with open('../corpora/buzzfeed/buzzfeed.html', 'w') as f:
    f.write(html)
# display(HTML(html))

In [331]:
soup = BeautifulSoup(html)
rule_ps = soup.find_all('p')[4:-1]
rule_ps_and_brs = soup.find_all(['p', 'br'])[4:-1]

In [271]:
display(HTML(str(rule_ps[0])))

In [650]:
from bs4 import BeautifulSoup, Tag
# Function to split a <b> tag with multiple <br/> into separate <b> tags
def split_b_tag(b_tag, tag_type='b'):
    def create_new_b_tag(current_contents):
        new_tag = soup.new_tag(tag_type)
        new_tag.extend(current_contents)
        return new_tag
    
    new_tags = []
    current_contents = []  # Temporary list to hold contents for each new <b> tag
    elements = list(b_tag.contents)
    for content in elements:
        # print(content)
        if isinstance(content, Tag) and content.name == 'br':
            if current_contents:  # Only create a <b> tag if there is content to include
                new_b_tag = create_new_b_tag(current_contents)
                new_tags.append(new_b_tag)
                new_tags.append(soup.new_tag('br'))
                current_contents = []  # Reset for the next <b> tag
        else:
            current_contents.append(content)
        # print(current_contents)            

    # Check if there is any remaining content to be added to a new <b> tag
    if current_contents:
        new_b_tag = create_new_b_tag(current_contents)
        new_tags.append(new_b_tag)
        new_tags.append(soup.new_tag('br'))

    return new_tags


def parse_blocks(p_list):
    data = {}
    current_title = None
    current_text = []

    # Function to flush current block into data
    def flush():
        if current_title is not None:
            full_text = ' '.join(current_text).strip()
            if current_title in data:
                data[current_title] += "\n" + full_text  # Append to existing title
            else:
                data[current_title] = full_text
            current_text.clear()

    # Iterate through each paragraph
    for p in p_list: # html_soup.find_all('p'):
        elements = list(p.children)
        expanded_elements_list = []
        for element in elements:
            if element.name == 'b':
                expanded_elements_list += split_b_tag(element)
            else:
                expanded_elements_list.append(element)
        elements = expanded_elements_list
        i = 0
        while i < len(elements):
            element = elements[i]
            if element.name == 'b' and (i == 0 or elements[i - 1].name == 'br'):
                flush()  # Flush previous block
                current_title = element.get_text(strip=True)
                # Extend the title to include all text up to ":"
                # current_title = full_title.split(":")[0] + ':' if ':' in full_title else full_title
            elif element.name == 'b':
                # If <b> is not the first element or after <br>, add its text to the current text block
                current_text.append(element.get_text(strip=True))
            elif element.name == 'br':
                # Flush the current block if <br> is found and start a new block if the next element is text or <b>
                if i + 1 < len(elements) and (isinstance(elements[i + 1], str) or elements[i + 1].name == 'b'):
                    flush()
            else:
                # Append any text or the text of any other element that is not <b>
                text = element if isinstance(element, str) else element.get_text(strip=True)
                if text:
                    current_text.append(text)
            i += 1
            
        # Flush after processing each paragraph to ensure no data is missed
        flush()

    # output_data = {}
    # for key, value
    return data

In [ ]:
soup = BeautifulSoup(html)
rule_ps = soup.find_all('p')[4:-1]
elements = list(rule_ps[1].children)
e = elements[117]
buzzfeed_rules = parse_blocks(rule_ps)
buzzfeed_rules_s = pd.Series(buzzfeed_rules).str.strip().str.replace(' +', ' ', regex=True)
buzzfeed_rules = buzzfeed_rules_s.to_dict()

import re
reparsed_buzzfeed_dict = {}
for key, value in buzzfeed_rules.items():
    if ':' in ' '.join(value.split()[:8]):
        b, e = value.split(':', maxsplit=1)
        key = key + ' ' + b
        key = re.sub(r'\s+', ' ', key).strip()
        value = e
    reparsed_buzzfeed_dict[key] = value

In [513]:
buzzfeed_rules_s = pd.Series(reparsed_buzzfeed_dict).str.strip()

In [516]:
(buzzfeed_rules_s
     .to_frame('rule_text')
     .reset_index()
     .rename(columns={'index': 'rule_title'})
     .to_csv('../corpora/buzzfeed/parsed-rules-df.csv')
)

# Mother Jones

In [532]:
import playwright
from playwright.async_api import async_playwright

In [545]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

In [546]:
t = 'https://www.motherjones.com/media/2020/02/mother-jones-style-guide/'
# r = requests.get(t)

In [547]:
await page.goto(t)

<Response url='https://www.motherjones.com/media/2020/02/mother-jones-style-guide/' request=<Request url='https://www.motherjones.com/media/2020/02/mother-jones-style-guide/' method='GET'>>

In [548]:
html_t = await page.content()

In [520]:
# mkdir ../corpora/mother_jones

In [550]:
with open('../corpora/mother_jones/all_rules.html', 'w') as f:
    f.write(html)

In [551]:
soup = BeautifulSoup(html)

In [552]:
headers = soup.find_all(['h3', 'p'])[5:]

In [561]:
pyperclip.copy('\n'.join(list(map(str, headers[:5]))))

In [569]:
e = soup.find_all(['h3', 'p'])[5:]

In [648]:
def get_hierarchy(tag):
    if tag.name == 'h3':
        return 1
    elif tag.name == 'p':
        span = tag.find('span')
        if ('style' in tag.attrs and 'padding-left: 40px;' in tag.attrs['style']):
            return 3
        # elif (span and 'class' in span.attrs and ('section-lead' in span.attrs['class'])):
            # return 3
        else:
            return 2
    return None

soup = BeautifulSoup(html)
results = []
current_title_path = []
current_text = []

for tag in soup.find_all(['h3', 'p'])[5:]:
    hierarchy_level = get_hierarchy(tag)
    strong_tag = tag.find('strong')
    if strong_tag:
        # Save text if collected and reset
        if current_title_path:
            results.append({
                'Title': current_title_path[-1],
                'Hierarchy Path': ' -> '.join(current_title_path[:-1]),
                'Text': '\n'.join(current_text)
            })
        current_text = []  # Reset text for the new block

        title_text = strong_tag.get_text(separator=" ", strip=True)
        # Adjust the current title path according to the hierarchy level
        # Reset for top level
        if hierarchy_level == 1:
            current_title_path = [title_text]  
        #
        # Append for deeper level            
        elif hierarchy_level > len(current_title_path):
            current_title_path.append(title_text) 
        #
        # Replace for same or shallower level            
        else:
            current_title_path = current_title_path[:hierarchy_level - 1] + [title_text] 
            
    # Append text to current block, excluding repeated title
    tag_text = tag.get_text(separator=" ", strip=True)
    tag_text = tag_text.replace(title_text, ' ', 1).strip() if strong_tag else tag_text.strip()
    if tag_text:
        current_text.append(tag_text)

# Append the last collected block if any
if current_title_path:
    results.append({
        'Title': current_title_path[-1],
        'Hierarchy Path': ' -> '.join(current_title_path[:-1]),
        'Text': ' '.join(current_text)
    })

In [649]:
results[-7]

{'Title': 'W',
 'Hierarchy Path': 'Preferred Spelling',
 'Text': 'Walmart war on terror Washington, DC, with commas and no periods wellbeing whoa , unless you insist on “woah,” but why would you insist? wifi winner-takes-all wiseguy WMD not WMDs wokeism words use quotes (e.g., the word “color” has two syllables)'}

In [652]:
results_df = pd.DataFrame(results[:-3])

In [656]:
results_df.rename(columns={
    'Title':'rule_title', 
    'Text': 'rule_text',
    'Hierarchy Path': 'rule_hierarchy'
}).to_csv('../corpora/mother_jones/mother_jones_parsed_rules.csv')